In [7]:
import requests

url = "https://apis.data.go.kr/B551011/GoCamping/basedList"


request_params = {
    "numOfRows": "50",
    "pageNo": "1",
    "MobileOS": "ETC",    
    "MobileApp": "camnavi",
    "servicekey" : "vXpamPPqXWsORdlj04YF6P%2FII0iMXCy2cV5WrUe78yyyLIH3JSQ5Fl82BQb0YzAJwwhceO%2Bxn%2Fae1OvcA7pXmg%3D%3D&",
    "_type": "xml"
    
}

res = requests.get(url, params=request_params)
print(res)

SSLError: HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /B551011/GoCamping/basedList?numOfRows=50&pageNo=1&MobileOS=ETC&MobileApp=camnavi&servicekey=vXpamPPqXWsORdlj04YF6P%252FII0iMXCy2cV5WrUe78yyyLIH3JSQ5Fl82BQb0YzAJwwhceO%252Bxn%252Fae1OvcA7pXmg%253D%253D%26&_type=xml (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_ILLEGAL_PARAMETER] sslv3 alert illegal parameter (_ssl.c:1028)')))

In [10]:
import subprocess

url = "https://apis.data.go.kr/B551011/GoCamping/basedList?numOfRows=50&pageNo=50&MobileOS=ETC&MobileApp=camnavi&serviceKey=vXpamPPqXWsORdlj04YF6P%2FII0iMXCy2cV5WrUe78yyyLIH3JSQ5Fl82BQb0YzAJwwhceO%2Bxn%2Fae1OvcA7pXmg%3D%3D&_type=xml"

result = subprocess.run(["curl", "-k", url], capture_output=True, text=True, encoding="utf-8")
print(result.stdout)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header>
  <resultCode>0000</resultCode>
  <resultMsg>OK</resultMsg>
</header><body><items>
  <item>
    <contentId>7424</contentId>
    <facltNm>관리도캠핑장</facltNm>
    <lineIntro></lineIntro>
    <intro></intro>
    <allar>3376</allar>
    <insrncAt>Y</insrncAt>
    <trsagntNo></trsagntNo>
    <bizrno></bizrno>
    <facltDivNm></facltDivNm>
    <mangeDivNm>직영</mangeDivNm>
    <mgcDiv></mgcDiv>
    <manageSttus>운영</manageSttus>
    <hvofBgnde></hvofBgnde>
    <hvofEnddle></hvofEnddle>
    <featureNm></featureNm>
    <induty>일반야영장</induty>
    <lctCl></lctCl>
    <doNm>전라북도</doNm>
    <sigunguNm>군산시</sigunguNm>
    <zipcode>54000</zipcode>
    <addr1>전북 군산시 옥도면 관리도길 60-19</addr1>
    <addr2></addr2>
    <mapX>126.3688876</mapX>
    <mapY>35.8177416</mapY>
    <direction></direction>
    <tel>063-454-2932</tel>
    <homepage></homepage>
    <resveUrl></resveUrl>
    <resveCl></resveCl>
    <manageNmpr>0</manageNmpr>
    <gnrl

In [21]:
import subprocess
import xml.etree.ElementTree as ET
import requests
import openpyxl
from openpyxl.drawing.image import Image as excelimage
from io import BytesIO


url = "https://apis.data.go.kr/B551011/GoCamping/basedList?numOfRows=50&pageNo=50&MobileOS=ETC&MobileApp=camnavi&serviceKey=vXpamPPqXWsORdlj04YF6P%2FII0iMXCy2cV5WrUe78yyyLIH3JSQ5Fl82BQb0YzAJwwhceO%2Bxn%2Fae1OvcA7pXmg%3D%3D&_type=xml"

result = subprocess.run(["curl", "-k", url], capture_output=True, text=True, encoding="utf-8")
root = ET.fromstring(result.stdout)
row = 2



wb = openpyxl.Workbook()
ws = wb.active
ws.title = "캠핑장정보"
ws.append(["캠핑장 아이디", "캠핑장명", "이미지", "캠핑장 지역", "주소", "홈페이지"])

items = root.findall(".//item")

for item in items :
    contentId = item.findtext("contentId", default="")
    facltNm = item.findtext("facltNm", default="")
    firstImageUrl = item.findtext("firstImageUrl", default="")
    doNm = item.findtext("doNm", default="")
    addr1 = item.findtext("addr1", default="")
    homepage = item.findtext("homepage", default="")

    ws.append([contentId, facltNm, "", doNm, addr1, homepage ])

    if firstImageUrl :
        try : 
            img_data = requests.get(firstImageUrl, timeout=10).content
            img_file = BytesIO(img_data)
            img = excelimage(img_file)
            img.width = 100
            img.height = 75
            ws.add_image(img, f"C{row}")
            ws.row_dimensions[row].height = 50
            
        except Exception as e :
            print(f"[경고] 이미지 다운로드 실패 : {firstImageUrl} -> {e} ")
    row += 1

    

ws.column_dimensions["B"].width = 30
ws.column_dimensions["C"].width = 20
ws.column_dimensions["D"].width = 15
ws.column_dimensions["E"].width = 50
ws.column_dimensions["F"].width = 50
wb.save("camping_info_2.xlsx")
print("🚗camping_info_2.xlsx 파일저장 완료!")


🚗camping_info_2.xlsx 파일저장 완료!
